# Holoviews live demo

This notebook is a demo on how we can concurrently visualise and run OC Time Driver simulations. 

This function is designed to work in parallel with a second notebook open. This second notebook should have an ubermag system initialised and it is where the Time Driver simulation is run. 

The system this function was built and tested on was a similar system to the Permalloy disk sample defined in the Ubermag demo. Please refer to the Ubermag demo for detail on how to initialise the system.

### Module Info

To concurrently visualise and run a Time Driver simulation we use common python modules but for the specialised tasks of monitoring directory events and handling a data pipeline we use the Watchdog and holoviews.streams.Pipe .


With Watchdog we can assign a directory to monitor. This directory will be where we have our simulation files produced and stored. Once we have assigned a directory to monitor we can create a visualisation pipeline with holoviews.streams.Pipe.


With holoviews.streams.Pipe we can pass formatted data from .omf files through a holoviews pipe where they are displayed onto an updating holoviews DynamicMap.

## Simulation

Firstly, we import the necessary modules

In [47]:
import discretisedfield as df
import xarray as xr
import holoviews as hv
import numpy as np
import time
import os
import panel as pn
import json

from holoviews.streams import Pipe
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

hv.extension("bokeh", logo=False)
pn.extension()

This is the class that contains the functionality for the live visualisation. We overwrite methods from the watchdog abstract class FileSystemEventHander.

In [ ]:
class DirectoryMonitorHandler(FileSystemEventHandler):
    def __init__(self, observer, stop_flag):
        self.observer = observer
        self.file_count = 0
        self.t = 0 # init with value = 0
        self.n = 1 # init with value = 1 to prevent division by 0 in process_file method
        self.stop_flag = stop_flag
        self.monitored_directories = self.get_existing_subdirectories()
        self.pipe1 = Pipe(data=[])
        self.pipe2 = Pipe(data=[])
        self.vector_dmap = hv.DynamicMap(hv.VectorField, streams=[self.pipe1]).opts(
            magnitude="mag", aspect=1
        )
        self.Image = hv.DynamicMap(hv.Image, streams=[self.pipe2]).opts(
            cmap="coolwarm", colorbar=True
        )
        self.plot = self.Image * self.vector_dmap
        self.widget()

    def get_existing_subdirectories(self):
        cwd = os.getcwd()
        list_dir = os.listdir(cwd)
        monitored_directories = {
            os.path.join(cwd, i)
            for i in list_dir
            if os.path.isdir(os.path.join(cwd, i))
        }
        return monitored_directories

    def on_created(self, event):
        if event.is_directory:
            print(f"New directory created: {event.src_path}")
            self.monitor_new_directory(event.src_path)
        else:
            print(f"New file detected: {event.src_path}")
            self.process_file(event.src_path)

    def monitor_new_directory(self, directory_path):
        if directory_path not in self.monitored_directories:
            self.monitored_directories.add(directory_path)
            print(f"Now monitoring: {directory_path}")

    def process_file(self, file_path):
        # Only process files with specific extensions
        if file_path.endswith(r".omf"):
            field = df.Field.from_file(rf"{file_path}")
            field_xr = field.to_xarray()
            # x_array for each vdim component
            xxa = field_xr.sel(vdims="x")  # arrow_x at each coordinate
            yxa = field_xr.sel(vdims="y")  # arrow_y
            zxa = field_xr.sel(vdims="z")  # arrow_z

            angle = np.arctan2(yxa, xxa)  # angle of vector line
            mag = np.sqrt(xxa**2 + yxa**2)  # magnitude of vector

            table = xr.Dataset(
                dict(angle=angle[:, :, 0], mag=mag[:, :, 0], W=zxa[:, :, 0])
            )  # xarray of processed data needed for hv visualisation

            self.pipe1.send(table)  # pipe for vectorfield
            self.pipe2.send(table["W"])  # pipe for scalar field

            self.file_count += 1
            self.check_observer()

            self.time_step.value = (self.t / self.n) * (self.file_count - 1)

        if file_path.endswith(r"m0.omf"):
            layout = pn.Row(
                self.plot, self.time_step
            )  
            # display one updating dmap that initially gets created with m0 file
            display(layout) 

        if file_path.endswith(r".json"):
            # read info.json to have the simulation time and number of files
            with open(file_path, 'r') as info:
                self.data = json.load(info)
            self.t = self.data["t"]
            self.n = self.data["n"]

    def widget(self):
        time_step = pn.indicators.Number(
            name="t:",
            format="{value:.2g}",
            title_size="10pt",
            font_size="10pt",
            default_color="white",
        )
        self.time_step = time_step

    def check_observer(self):
        if self.file_count >= self.n + 1:
            print("All files produced, stopping monitoring...")
            self.observer.stop()
            self.stop_flag[0] = True

The function below initialises the Watchdog Observer to monitor the desired directory.  

In [ ]:
def main(working_directory=os.getcwd()):
    """
    Function to plot hv plots whilst simultaneously running a Time Driver simulation

    Parameters:
    working_directory: The directory we want Watchdog to monitor. Defaulted to the current working directory
    """

    # stop flag used to terminate thread when all files are produced.
    stop_flag = [False]

    # Set up the observer
    observer = Observer()
    handler = DirectoryMonitorHandler(observer, stop_flag)

    # Schedule the observer to monitor the working directory
    observer.schedule(handler, working_directory, recursive=True)
    print(f"Monitoring: {working_directory}")

    # Start the observer
    observer.start()

    try:
        while not stop_flag[0]:
            time.sleep(5)  # Keeps the script running
    except KeyboardInterrupt:
        print("Stopping monitoring...")
    finally:
        observer.stop()

    observer.join()

Run the cell below first to begin directory monitoring and then run the Time Driver simulation in your other notebook.


**Remember** to specify path of the directory you want monitored if your simulation files aren't being produced in the current working directory.

In [ ]:
main()

Monitoring: C:\Users\Noahd\IP code
New directory created: C:\Users\Noahd\IP code\test_1\drive-104
Now monitoring: C:\Users\Noahd\IP code\test_1\drive-104
New file detected: C:\Users\Noahd\IP code\test_1\drive-104\m0.omf


BokehModel(combine_events=True, render_bundle={'docs_json': {'395b53fb-3f0c-4419-a8ef-eb0a553b443d': {'version…

New file detected: C:\Users\Noahd\IP code\test_1\drive-104\test_1.mif
New file detected: C:\Users\Noahd\IP code\test_1\drive-104\info.json
New file detected: C:\Users\Noahd\IP code\test_1\drive-104\test-noahs-laptop-18416
New file detected: C:\Users\Noahd\IP code\test_1\drive-104\test_1.restart-noahs-laptop-18416-oxs-checkpoint.tmp
New file detected: C:\Users\Noahd\IP code\test_1\drive-104\test-noahs-laptop-18416
New file detected: C:\Users\Noahd\IP code\test_1\drive-104\test_1.restart-noahs-laptop-18416-oxs-checkpoint.backup
New file detected: C:\Users\Noahd\IP code\test_1\drive-104\test_1.odt
New file detected: C:\Users\Noahd\IP code\test_1\drive-104\test_1-Oxs_TimeDriver-Magnetization-00-0003274.omf
New file detected: C:\Users\Noahd\IP code\test_1\drive-104\test_1-Oxs_TimeDriver-Magnetization-01-0006548.omf
New file detected: C:\Users\Noahd\IP code\test_1\drive-104\test_1-Oxs_TimeDriver-Magnetization-02-0009822.omf
New file detected: C:\Users\Noahd\IP code\test_1\drive-104\test_1-Ox